In [1]:
import pandas as pd
import numpy as np

Documentation

Content based recommendation fundamentals: https://www.linkedin.com/pulse/content-based-recommender-engine-under-hood-venkat-raman 

Content base recommendation system: https://towardsdatascience.com/building-a-content-based-book-recommendation-engine-9fd4d57a4da

Sparse matrix recommendation system: https://towardsdatascience.com/building-my-own-2021-book-recommendation-engine-903ea10d5021


In [3]:
def get_datasets():
    goodbooks_dataset_path = "/home/joacopolo/Documents/software/python/draft-projects/books-recommender/goodbooks-10k"
    books = pd.read_csv(goodbooks_dataset_path+"/books.csv")
    book_tags = pd.read_csv(goodbooks_dataset_path+"/book_tags.csv")
    ratings = pd.read_csv(goodbooks_dataset_path+"/ratings.csv")
    tags = pd.read_csv(goodbooks_dataset_path+"/tags.csv")
    to_read = pd.read_csv(goodbooks_dataset_path+"/to_read.csv")
    return (books, book_tags, ratings, tags, to_read)

In [ ]:
books, book_tags, ratings, tags, to_read = get_datasets()

In [4]:
n_users = ratings.user_id.unique().shape[0]
n_books = ratings.book_id.unique().shape[0]
ratings_matrix = np.zeros((n_users, n_books))



In [5]:
for row in ratings.itertuples():
    # row[1] - 1 is the user id readjusted to start by index 0
    # row[2] - 1 is the item id readjusted to start by index 0
    ratings_matrix[row[1]-1, row[2]-1] = row[3]


In [14]:
nonzeros = ratings_matrix.nonzero()
    

In [16]:
sample_row, sample_col = ratings_matrix.nonzero()

In [18]:
#takes sample row and col from the same index and its guaranteed to be non zero
n = 1
row = sample_row[n] 
col = sample_col[n]
ratings_matrix[row, col]

4.0

In [19]:
from SGD import SGDMF

In [20]:
sgd = SGDMF(ratings_matrix)

In [21]:
sgd.train()

/home/joacopolo/Documents/software/python/draft-projects/books-recommender/book-films-recommendations/SGD.py:47: RuntimeWarning: overflow encountered in multiply
  (error * self.user_vecs[u, :] - \
/home/joacopolo/Documents/software/python/draft-projects/books-recommender/book-films-recommendations/SGD.py:44: RuntimeWarning: overflow encountered in multiply
  (error * self.item_vecs[i, :] - \


Finished epoch 1 of 10
Finished epoch 2 of 10
Finished epoch 3 of 10
Finished epoch 4 of 10
Finished epoch 5 of 10
Finished epoch 6 of 10
Finished epoch 7 of 10
Finished epoch 8 of 10
Finished epoch 9 of 10
Finished epoch 10 of 10


In [22]:
import pickle
with open('sgd_predictor.pkl', 'wb') as sgd_predictor_file:
    pickle.dump(sgd, sgd_predictor_file)

In [28]:
sgd.user_vecs.shape #returns (53424, 40)
sgd.item_vecs.T.shape #returns (40, 10000)
ratings_matrix.shape #returns (53424, 10000)

#so user_vecs dot (items_vecs transposed) has the shape of the ratings matrix

(40, 10000)